# Trabalho



In [8]:
import pandas as pd
import numpy as np
import sympy as sym
import scipy as sp

def read_graph_from_csv(csv_file, directed=True,df=None):
    if df is None:
        # Read CSV file into a pandas DataFrame
        df = pd.read_csv(csv_file, header=None, names=['Source', 'Destination', 'Cost'])

    # Get unique nodes
    nodes = sorted(set(df['Source']) | set(df['Destination']))

    # Create adjacency matrix A and Cost matrix C
    num_nodes = len(nodes)
    A = np.zeros((num_nodes, num_nodes), dtype=int)
    C = np.zeros((num_nodes, num_nodes), dtype=float)

    # Fill adjacency matrix and Cost matrix
    for _, row in df.iterrows():
        Source_index = nodes.index(row['Source'])
        destination_index = nodes.index(row['Destination'])
        A[Source_index, destination_index] = 1
        C[Source_index, destination_index] = row['Cost']
        if not directed:
            # For undirected graph, add edges in both directions
            A[destination_index, Source_index] = 1
            C[destination_index, Source_index] = row['Cost']

    return A, C

In [10]:
import matplotlib.pyplot as plt
import networkx as nx

def plot_graph_from_csv(csv_file, directed=True,df=None):
    if df is None:
        # Read CSV file into a pandas DataFrame
        df = pd.read_csv(csv_file, header=None, names=['Source', 'Destination', 'Cost'])

    # Create a directed or undirected graph
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()

    # Add edges to the graph
    for _, row in df.iterrows():
        G.add_edge(row['Source'], row['Destination'], weight=row['Cost'])

    # Position nodes using the spring layout algorithm
    pos = nx.spring_layout(G)

    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=500)

    # Draw edges
    nx.draw_networkx_edges(G, pos, arrows=directed)

    # Draw edge labels
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

    # Draw node labels
    nx.draw_networkx_labels(G, pos)

    # Show plot
    plt.title('Graph')
    plt.axis('off')
    plt.show()

In [11]:
def generate_random_graph(num_vertices, num_edges_per_vertex, RandomCostFunction, randomCostArgs=(),randomCostKWArgs={}):
    G = nx.Graph()
    # Add vertices
    G.add_nodes_from(range(num_vertices))
    
    # Add edges
    for v in range(num_vertices):
        # Generate random edges for each vertex
        edges = np.random.choice(range(num_vertices), size=num_edges_per_vertex, replace=False)
        for edge in edges:
            # Ensure not to add self-loops
            if edge != v:
                # Generate random Cost for each edge
                Cost = RandomCostFunction(*randomCostArgs,**randomCostKWArgs)  # Adjust range as per your requirements
                G.add_edge(v, edge, weight=Cost)

    # Extract edges and their attributes (Cost)
    edges_data = [(Source, target, data['weight']) for Source, target, data in G.edges(data=True)]
    
    # Create DataFrame from edges data
    df = pd.DataFrame(edges_data, columns=['Source', 'Destination', 'Cost'])

    return df

In [181]:
class Grafo:
    def __init__(self,numeroNos,nosEnlace,custo,custoArgs=(),custoKwargs={}):
        self.enlaces = generate_random_graph(numeroNos,nosEnlace,custo,custoArgs,custoKwargs)
        self.A, self.C = read_graph_from_csv('', directed=True,df=df)
        self.fontes, self.destinos = ([],[])
        self.nullSpace = None
        self.mudou = True
        self.nullFlowConservationMatrix = None
        self.contractFlowConservationMatrix = None
        self.flowConservationMatrix = None
        
        
    def VerGrafo(self):
        plot_graph_from_csv('',directed=False,df=self.enlaces)

    def ImportarGrafo(self,csv_file):
        self.A, self.C = read_graph_from_csv(csv_file, directed=True,df=self.enlaces)
        self.enlaces = pd.read_csv(csv_file, header=None, names=['Source', 'Destination', 'Cost'])
        self.mudou = True
        
    def ExportarGrafo(self,filename):
        self.enlaces.to_csv(filename)

    def AdicionarFonte(self,fonte):
        self.fontes.append(fonte)

    def AdicionarDestino(self,destino):
        self.fontes.append(destino)

    def MapearVariaveisLivres(self):
        if not self.mudou:
            self.nullSpace.shape[1]
        self.mudou = False
        
        self.ValidarListaEnlaces()
        # -------- CASO DIRECIONADO ------------------
        # Transformação de enlaces de grafos não direcionados para enlaces de grafos direcionados
        #directedEdges = pd.DataFrame(
        #    np.concatenate(
        #        [self.enlaces,self.enlaces[['Destination','Source','Cost']]]
        #        )
        #    ,columns=['Source','Destination','Cost'])
        #
        # Calculo da matriz de conservação de fluxos
        #flowConservationMatrix = np.stack(
        #    [(1*(directedEdges['Source']==i))-(1*(directedEdges['Destination']==i)) for i in range(nos)]
        #)
        # ----------------------------------------------

        # Caso não direcionado
        self.flowConservationMatrix = np.stack(
            [
                (1*(self.enlaces['Source']==i))+1*(self.enlaces['Destination']==i) 
                 for i in set(self.enlaces['Source'].unique()).union(set(self.enlaces['Destination'].unique()))]
        )
        
        # Calculo dos índices
        filtroFonte = pd.Series(np.arange(self.flowConservationMatrix.shape[0])).isin(self.fontes)
        filtroDestino = pd.Series(np.arange(self.flowConservationMatrix.shape[0])).isin(self.destinos)
        filtro = ~(np.array(filtroFonte)|np.array(filtroDestino))

        # Separa a matriz de conservação de fluxo em dois: Uma para os nós intermediários e outra
        # para os nós fonte/destino
        self.nullFlowConservationMatrix = self.flowConservationMatrix[filtro,:]
        self.contractFlowConservationMatrix = self.flowConservationMatrix[~filtro,:]
        self.flowConservationMatrix = None
        
        # Calcula-se o espaço nulo da matriz
        self.nullSpace = sp.linalg.null_space(self.nullFlowConservationMatrix)

        # Retorna o número de variáveis livres disponível
        return self.nullSpace.shape[1]
    
    def CalcularFluxoReal(self,fluxoLivre):
        if self.nullSpace is None:
            raise Exception("Deve-se rodar MapearVariaveisLivres antes de calcular o fluxo")
            
        if fluxoLivre.shape[0] != self.nullSpace.shape[1]:
            raise Exception("A variável de fluxo livre não tem a mesma dimensão que o número de variáveis livres")
        
        return self.nullSpace@fluxoLivre

    def CustoEnlaces(self):
        return self.enlaces['Cost']

    def FluxoContratos(self):
        return self.contractFlowConservationMatrix 

    def FluxoNulo(self):
        return self.nullFlowConservationMatrix 
        
    def ValidarListaEnlaces(self):
        """
        Não pode existir na lista de enlaces não direcionados entradas (x,y) e (y,x), já que são o mesmo enlace
        """
        for i in self.enlaces['Source'].unique():
            for j in self.enlaces['Destination'].unique():
                if self.enlaces[(self.enlaces['Destination'] == i)&(self.enlaces['Source'])==j].shape[0] > 0:
                    raise Exception("Lista mal definida. Não pode existir na lista de enlaces não direcionados entradas (x,y) e (y,x), já que são o mesmo enlace")

# Mapeamento de espaço

In [189]:
grafo.MapearVariaveisLivres()

79


## Função custo
Deve conter: 
* Soma dos custos pelo fluxo passando no enlace
* Soma dos fluxos fora do contrato

In [206]:
# Cria um novo grafo
grafo = Grafo(20,6,np.random.randint,(1,10))

# Adiciona fonte e destino para fluxos de dados
for fonte in [4,5]:
    grafo.AdicionarFonte(fonte)
for destino in [7,15]:
    grafo.AdicionarDestino(destino)

# Define o fluxo de contrato (positivo -> entrando, negativo -> saindo)
fluxoContrato = np.array([[100],[300],[-300],[-100]])

# Mapeia o número de variáveis livres que podem ser modificadas
Nl = grafo.MapearVariaveisLivres()

# Gera o vetor com o número de variáveis 
x = np.zeros((Nl,1))

# Calcula a função de custo em duas etapas:
# 1 - Calcula o fluxo real
xReal = grafo.CalcularFluxoReal(x)
# 2 - Calcula o custo em função dos enlaces
custo = (grafo.CustoEnlaces().T@xReal)+np.sum((grafo.FluxoContratos()@xReal - fluxoContrato)**2)

In [208]:
custo

array([200000.])